### Web Crawler que coleta codigos de cid das doenças  

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv

In [2]:
# Inicie o ChromeDriver sem especificar o caminho (assumindo que o ChromeDriver está no PATH)
driver = webdriver.Chrome()

In [3]:

try:
    # Abre o site do CID-10
    driver.get('https://pebmed.com.br/cid10/')

    # Localiza o campo de busca e insere o termo "joelho"
    search_box = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, 'campo-busca'))
    )
    search_box.send_keys('menisco')
    search_box.send_keys(Keys.RETURN)

    time.sleep(3)  # Aguarda o carregamento dos resultados

    previous_page_content = ""

    # Abre o arquivo CSV para escrever os resultados
    with open('cids_menisco.csv', 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['CID', 'DESCRICAO']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        while True:
            # Coleta os resultados da página
            codes = driver.find_elements(By.XPATH, '//div[@class="code__cid10"]')
            descriptions = driver.find_elements(By.XPATH, '//div[@class="description__cid10"]')

            current_page_content = "".join([code.text.strip() + description.text.strip() for code, description in zip(codes, descriptions)])

            # Verifica se o conteúdo mudou, se não, sai do loop
            if current_page_content == previous_page_content:
                print("Conteúdo repetido, terminando...")
                break

            # Atualiza o conteúdo da página anterior
            previous_page_content = current_page_content

            # Escreve os resultados no CSV
            for code, description in zip(codes, descriptions):
                writer.writerow({'CID': code.text.strip(), 'DESCRICAO': description.text.strip()})

            try:
                # Aguarda o botão "Próxima página" estar presente
                next_button = WebDriverWait(driver, 15).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'button.next'))
                )

                # Verifica se o botão está habilitado
                if 'disabled' in next_button.get_attribute('class'):
                    break  # Sai do loop se não houver próxima página

                # Usa JavaScript para clicar no botão
                driver.execute_script("arguments[0].click();", next_button)

                # Aguarda que os novos dados carreguem
                time.sleep(3)  # Ajuste o tempo conforme necessário

            except Exception as e:
                print(f"Erro: {e}")
                break  # Sai do loop se o botão não puder ser clicado ou ocorrer outro erro

finally:
    # Fecha o navegador
    driver.quit()

Conteúdo repetido, terminando...
